In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
import numpy as np
import string
from string import ascii_lowercase
import multiway_decision_tree as mdt
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.svm import LinearSVC
import pickle

In [3]:
class Split_actor:
    def __init__( self, attr ):
        self.attr = attr
        
    def get_attr( self ):
        return self.attr
    
    def split( self, tst_pt, ancestors ):
        if len( ancestors ) == 0:
            mask = input( "Start  : " )
        else:
            mask = input( "Guess " + str( self.attr ) + ": " )
        
        # Splitting a single point so the dictionary is trivial
        split_dict = {}
        split_dict[ mask ] = ( [ 0 ], [ mask ] )
        
        return split_dict
        
    def default_predict( self, tst_pt, ancestors ):
        return [ None ]
    
class Leaf_actor:
    def __init__( self, data ):
        self.data = data
        
    def predict( self, tst_pt, ancestors ):
        return [ self.data[0] ]
        
    def default_predict( self, tst_pt, ancestors ):
        return [ None ]

def get_size( trn_pts ):
    return len( trn_pts )

def is_pure_enough( trn_pts ):
    if len( trn_pts ) < 2:
        return True
    else:
        return False

def get_leaf_actor( trn_pts, ancestors ):
    return Leaf_actor( data = trn_pts )

# See the following StackOverflow conversation on some alternate ways to compute entropy
# https://stackoverflow.com/questions/15450192/fastest-way-to-compute-entropy-in-python
def get_entropy( counts ):    
    # Why include elements that do not appear at all?
    assert np.min( counts ) > 0, "Elements with zero or negative counts detected"
    
    num_elements = counts.sum()
    
    # A singular or empty set has zero entropy
    if num_elements <= 1:
        print( f"warning: { num_elements } elements in total." )
        return 0
    
    proportions = counts / num_elements
    
    return np.sum( proportions * np.log2( counts ) )  

def get_mask( char_list, word ):
    mask = []
    
    for c in word:
        if c in char_list:
            mask.append( c )
        else:
            mask.append( '_' )
    
    return ' '.join( mask )

def try_attr( char_list, words ):
    split_dict = {}
    count_dict = {}
    
    for word in words:
        mask = get_mask( char_list, word )
        
        if mask not in split_dict:
            split_dict[ mask ] = []
            count_dict[ mask ] = 0
            
        split_dict[ mask ].append( word )
        count_dict[ mask ] += 1

    entropy = get_entropy( np.array( list( count_dict.values() ) ) )
    
    return ( entropy, split_dict )

def get_split_actor( trn_pts, ancestors ):
    
    # If this is the root node, we must split on word length
    if len( ancestors ) == 0:
        ( entropy, split_dict ) = try_attr( [], trn_pts )
        return ( Split_actor( "len" ), split_dict )
        
    # Exclude the first element of the ancestry since that is the length split
    ancestor_attr = [ split[0] for split in ancestors ][1:]
    
    best_entropy = np.inf
    best_attr = ''
    best_split_dict = None
    
    for c in [ char for char in ascii_lowercase if char not in ancestor_attr ]:
        char_list = ancestor_attr.copy()
        char_list.append( c )
        ( entropy, split_dict ) = try_attr( char_list, trn_pts )
#         print( f"{c}({entropy})", end = ", " )
        if entropy < best_entropy:
            best_entropy = entropy
            best_attr = c
            best_split_dict = split_dict
    
#     print()
#     print( f"-->{best_attr}({best_entropy})\n" )
    
    return ( Split_actor( best_attr ), best_split_dict )

In [4]:
f = open( "base", 'r' )
words = f.read().split( '\n' )[:-1]        # Omit the last line since it is empty
f.close()

In [8]:
max_depth = 15
dtID3 = mdt.Tree( min_leaf_size = 1, max_depth = max_depth )
dtID3.train( words, get_split_actor, get_leaf_actor, is_pure_enough, get_size, verbose = True )

with open( "dt.mdl", "wb" ) as f:
    pickle.dump( dtID3, f )

root
└───len
    ├───e
    │   ├───a
    │   │   ├───g
    │   │   │   ├───b
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───█
    │   │   │   ├───█
    │   │   │   └───█
    │   │   ├───o
    │   │   │   ├───g
    │   │   │   │   ├───█
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───b
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───c
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───i
    │   │   │   │   ├───█
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───█
    │   │   │   ├───█
    │   │   │   └───c
    │   │   │       ├───█
    │   │   │       └───█
    │   │   ├───█
    │   │   ├───█
    │   │   ├───c
    │   │   │   ├───█
    │   │   │   └───█
    │   │   ├───n
    │   │   │   ├───l
    │   │   │   │   ├───h
    │   │   │   │   │   ├───k
    │   │   │   │   │   │   ├───b
    │   │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   │   └───█
    │   │   │   │

    │   │   │   ├───i
    │   │   │   │   ├───█
    │   │   │   │   ├───d
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   └───d
    │   │   │   │       ├───█
    │   │   │   │       └───█
    │   │   │   ├───a
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───a
    │   │   │   │   ├───h
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   └───█
    │   │   │   ├───█
    │   │   │   └───a
    │   │   │       ├───█
    │   │   │       └───█
    │   │   ├───t
    │   │   │   ├───a
    │   │   │   │   ├───f
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   ├───c
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───l
    │   │   │   │   │       ├───█
    │   │   │   │   │       └───█
    │   │   │   │   └───o
    │   │   │   │       ├───█
    │   │   │   │       └───█
    │   │   │   ├───o
    │   │   │   │   ├───n
    │   │   │   │   │   ├───a
    │   │   │   │   │   │   ├───

    │   │   ├───t
    │   │   │   ├───r
    │   │   │   │   ├───n
    │   │   │   │   │   ├───a
    │   │   │   │   │   │   ├───b
    │   │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   │   └───█
    │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   └───█
    │   │   │   │   │   ├───█
    │   │   │   │   │   ├───a
    │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   └───j
    │   │   │   │   │   │       ├───█
    │   │   │   │   │   │       └───█
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   ├───a
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   ├───█
    │   │   │   │   ├───c
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   └───a
    │   │   │   │       ├───c
    │   │   │   │       │   ├───█
    │   │   │   │       │   └───█
    │   │   │   │       ├───█
    │   │   │   │       └───█
    │   │   │   ├───r
    │   │   │   │   ├───

    │   │   │   │       │   ├───c
    │   │   │   │       │   │   ├───█
    │   │   │   │       │   │   └───f
    │   │   │   │       │   │       ├───█
    │   │   │   │       │   │       └───g
    │   │   │   │       │   │           ├───█
    │   │   │   │       │   │           └───█
    │   │   │   │       │   └───█
    │   │   │   │       └───█
    │   │   │   ├───g
    │   │   │   │   ├───p
    │   │   │   │   │   ├───a
    │   │   │   │   │   │   ├───c
    │   │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   │   └───█
    │   │   │   │   │   │   ├───m
    │   │   │   │   │   │   │   ├───b
    │   │   │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   │   │   └───█
    │   │   │   │   │   │   │   └───d
    │   │   │   │   │   │   │       ├───█
    │   │   │   │   │   │   │       └───█
    │   │   │   │   │   │   └───d
    │   │   │   │   │   │       ├───█
    │   │   │   │   │   │       └───█
    │   │   │   │   │   ├───a
    │   │   │   │   │   │   ├───c
    │   │ 

    │   │   │   │   ├───c
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   └───█
    │   │   │   ├───f
    │   │   │   │   ├───█
    │   │   │   │   ├───d
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   └───█
    │   │   │   ├───█
    │   │   │   ├───█
    │   │   │   └───█
    │   │   ├───a
    │   │   │   ├───b
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───█
    │   │   │   ├───d
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───n
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───█
    │   │   │   ├───█
    │   │   │   └───█
    │   │   ├───r
    │   │   │   ├───█
    │   │   │   ├───█
    │   │   │   ├───█
    │   │   │   ├───l
    │   │   │   │   ├───█
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───a
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   └───█
    │   │   ├───a
    │   │   │   ├───█
    │   │   │ 

    │   │   │   ├───a
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───c
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───█
    │   │   │   └───█
    │   │   ├───a
    │   │   │   ├───█
    │   │   │   ├───█
    │   │   │   ├───█
    │   │   │   ├───█
    │   │   │   └───█
    │   │   ├───b
    │   │   │   ├───█
    │   │   │   └───█
    │   │   ├───a
    │   │   │   ├───█
    │   │   │   └───█
    │   │   ├───█
    │   │   ├───█
    │   │   ├───█
    │   │   ├───a
    │   │   │   ├───█
    │   │   │   └───█
    │   │   ├───o
    │   │   │   ├───a
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───█
    │   │   │   ├───g
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   └───█
    │   │   ├───█
    │   │   ├───a
    │   │   │   ├───█
    │   │   │   └───█
    │   │   ├───a
    │   │   │   ├───█
    │   │   │   └───█
    │   │   ├───█
    │   │   ├───n
    │   │   │   ├───█
    │   │   │   ├───a
    │   

    │   ├───o
    │   │   ├───r
    │   │   │   ├───a
    │   │   │   │   ├───█
    │   │   │   │   ├───█
    │   │   │   │   ├───g
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   └───█
    │   │   │   ├───f
    │   │   │   │   ├───a
    │   │   │   │   │   ├───b
    │   │   │   │   │   │   ├───█
    │   │   │   │   │   │   └───█
    │   │   │   │   │   └───█
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───l
    │   │   │   │   ├───█
    │   │   │   │   ├───█
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───a
    │   │   │   │   ├───b
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   └───h
    │   │   │   │       ├───█
    │   │   │   │       └───█
    │   │   │   └───█
    │   │   ├───i
    │   │   │   ├───a
    │   │   │   │   ├───s
    │   │   │   │   │   ├───█
    │   │   │   │   │   ├───█
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   ├───n


    │   │   │   ├───a
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   └───a
    │   │   │       ├───█
    │   │   │       └───█
    │   │   ├───a
    │   │   │   ├───b
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   └───g
    │   │   │       ├───█
    │   │   │       └───█
    │   │   ├───d
    │   │   │   ├───p
    │   │   │   │   ├───b
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   ├───a
    │   │   │   │   ├───█
    │   │   │   │   └───█
    │   │   │   └───█
    │   │   ├───t
    │   │   │   ├───d
    │   │   │   │   ├───a
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   ├───i
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │   │   │   │   └───a
    │   │   │   │       ├───█
    │   │   │   │       └───█
    │   │   │   ├───a
    │   │   │   │   ├───c
    │   │   │   │   │   ├───█
    │   │   │   │   │   └───█
    │ 

    │   ├───a
    │   │   ├───█
    │   │   └───█
    │   ├───█
    │   ├───█
    │   ├───█
    │   ├───█
    │   ├───a
    │   │   ├───█
    │   │   └───█
    │   ├───c
    │   │   ├───█
    │   │   └───█
    │   ├───█
    │   ├───█
    │   ├───d
    │   │   ├───█
    │   │   └───█
    │   ├───█
    │   ├───█
    │   ├───█
    │   ├───i
    │   │   ├───█
    │   │   └───█
    │   ├───c
    │   │   ├───█
    │   │   └───█
    │   ├───█
    │   ├───█
    │   └───█
    ├───e
    │   ├───a
    │   │   ├───c
    │   │   │   ├───█
    │   │   │   └───█
    │   │   ├───█
    │   │   ├───b
    │   │   │   ├───█
    │   │   │   └───█
    │   │   ├───c
    │   │   │   ├───█
    │   │   │   ├───█
    │   │   │   ├───█
    │   │   │   └───█
    │   │   ├───█
    │   │   ├───c
    │   │   │   ├───█
    │   │   │   └───█
    │   │   ├───█
    │   │   ├───█
    │   │   ├───█
    │   │   ├───█
    │   │   └───█
    │   ├───i
    │   │   ├───█
    │   │   ├───a
    │   │   │   ├───█
    │   │   │   └─

In [7]:
dtID3.predict( [ [] ] )

Start  : _ _ _ _ _ _ _ _
Guess e: _ _ _ _ e _ _ _
Guess r: _ _ _ _ e _ _ _
Guess n: _ _ _ _ e _ _ _


['academic']

In [9]:
def preprocess_data( data ):
    X_raw = data[:,:-1]
    y_raw = data[:,-1]
    
    X = np.cumprod( np.flip( 2 * X_raw - 1 , axis = 1 ), axis = 1 )
    y = np.where( y_raw > 0, 1, -1 )
    
    return ( X, y )

In [10]:
data = np.loadtxt( "APUF_CRP.txt" )

data_trn, data_tst = train_test_split( data, train_size = 10000 )

In [11]:
( X, y ) = preprocess_data( data_trn )
( X_t, y_t ) = preprocess_data( data_tst )

clf = LinearSVC( loss = "hinge" )
clf.fit( X, y )

w = clf.coef_.reshape( (32,) )
b = clf.intercept_
temp = X_t.dot(w) + b
y_t_pred = np.where( temp > 0, 1, -1 )

np.average( y_t == y_t_pred )

C:\Users\purus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.9947111111111111

In [12]:
class PUF_split_actor:
    def __init__( self, attr, pop_label ):
        self.attr = attr
        self.pop_label = pop_label
        
    def get_attr( self ):
        return self.attr
    
    def split( self, tst_pts, ancestors ):
        split_dict = {}
        
#         print( len( tst_pts ), end = '' )
        
        for val in np.unique( tst_pts[ :, self.attr ] ):
            idx = np.where( tst_pts[ :, self.attr ] == val )[0].tolist()
            split_dict[ val ] = ( idx, tst_pts[ idx, : ]  )
#             print( "[",idx,"]", end = '' )
        
#         print()
        return split_dict
        
    def default_predict( self, tst_pts, ancestors ):
        preds = []
        
        for x in tst_pts:
            preds.append( self.pop_label )
        
        return preds
    
class PUF_leaf_actor:
    def __init__( self, pop_label ):
        self.pop_label = pop_label
        
    def predict( self, tst_pts, ancestors ):
        preds = []
        
        for x in tst_pts:
            preds.append( self.pop_label )
        
        return preds
        
    def default_predict( self, tst_pt, ancestors ):
        return self.predict( tst_pts, ancestors )

def get_PUF_size( trn_pts ):
    return len( trn_pts[ 1 ] )

def is_PUF_pure_enough( trn_pts ):
    labels = trn_pts[ 1 ]
    # Pure enough for us if the labels are more than 99% pure
    # Since labels are +1/-1 valued this will happen exactly
    # when the average label has a magnitude greater than
    # 0.99 - (1 - 0.99)) = 0.98
    # Similarly, to get 95% purity, we should ask for the
    # magtinude to be greater than 0.95 - (1 - 0.95) = 0.90
    if np.abs( np.mean( labels ) ) > 0.98:
        return True
    else:
        return False

def get_PUF_leaf_actor( trn_pts, ancestors ):
    labels = trn_pts[ 1 ]
    ( classes, counts ) = np.unique( labels, return_counts = True )    
    return PUF_leaf_actor( pop_label = classes[ np.argmax( counts ) ] )

# See the following StackOverflow conversation on some alternate ways to compute entropy
# https://stackoverflow.com/questions/15450192/fastest-way-to-compute-entropy-in-python
def get_PUF_entropy( labels ):    
    
    _, counts = np.unique( labels, return_counts = True )
    
    # Why include elements that do not appear at all?
    assert np.min( counts ) > 0, "Elements with zero or negative counts detected"
    
    num_elements = counts.sum()
    
    # A singular or empty set has zero entropy
    if num_elements <= 1:
#         print( f"warning: { num_elements } elements in total" )
        return 0
    
    proportions = counts / num_elements
    
    return -np.sum( proportions * np.log2( proportions ) )

def try_PUF_attr( a, X, y ):
    n = len( y )
    assert n == X.shape[ 0 ], "Mismatch in number of features and number of labels"
    
    split_dict = {}
    entropy = 0
        
    for val in np.unique( X[ :, a ] ):
        # Find out who all would go to this child
        idx = np.where( X[ :, a ] == val )[0].tolist()
        split_dict[ val ] = ( X[ idx, : ], y[ idx ] )
        # Calculate the entropy of the child
        entropy += len(idx) / n * get_PUF_entropy( y[ idx ] )
    
    return ( entropy, split_dict )

def get_PUF_split_actor( trn_pts, ancestors ):
    ( X, y ) = trn_pts
    ( n, d ) = X.shape
    
    # Exclude the attribute used by my parent to split my siblings and me
    if len( ancestors ) < 1:
        parent_attr = -1
    else:
        parent_attr = ancestors[-1][0]
    
    best_entropy = np.inf
    best_attr = -1
    best_split_dict = None
    
    for a in [ attr for attr in np.arange( d ) if not attr == parent_attr ]:
        ( entropy, split_dict ) = try_PUF_attr( a, X, y )
#         print( f"{a}({entropy})", end = ", " )
        if entropy < best_entropy:
            best_entropy = entropy
            best_attr = a
            best_split_dict = split_dict
    
#     print()
    
#     print( f"\n\n\n{best_attr}({best_entropy})" )
    ( classes, counts ) = np.unique( y, return_counts = True )
#     print( f"\n\n\n{ counts } --> ", end = '' )
    for ( XX, yy ) in best_split_dict.values():
        ( _, cc ) = np.unique( yy, return_counts = True )
#         print( f"{ cc }, ", end = '' )
    
    pop_label = classes[ np.argmax( counts ) ]
    return ( PUF_split_actor( best_attr, pop_label ), best_split_dict )

In [16]:
max_depth = 15

dt_PUF = mdt.Tree( min_leaf_size = 1, max_depth = max_depth )
dt_PUF.train( ( X, y ), get_PUF_split_actor, get_PUF_leaf_actor, is_PUF_pure_enough, get_PUF_size )

with open( "dt_PUF.mdl", "wb" ) as f:
    pickle.dump( dt_PUF, f )

In [17]:
# Training accuracy
y_pred = np.array( dt_PUF.predict( X ) )
np.average( y == y_pred )

0.9992

In [18]:
# Test accuracy
y_t_pred = np.array( dt_PUF.predict( X_t ) )
np.average( y_t == y_t_pred )

0.8174888888888889